In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from pandas.plotting import scatter_matrix
from matplotlib import pyplot as plt
from sklearn import tree
#import pydotplus
import collections
from IPython.display import Image  
#import pydotplus
from sklearn.tree import export_graphviz

In [4]:
import copy

In [5]:
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [26]:
df = pd.read_stata("Econ_484_data_final.dta")

In [27]:
# df.isin(["''"]).sum(axis=1).sum()

In [28]:
df = df.drop("DAYSWAIT_CHRON", axis=1)

In [29]:
df = df.dropna(axis=0, how='any')
df= df.drop("PERM_STATE", axis=1)

In [30]:
df['year'] = pd.DatetimeIndex(df["TX_DATE"]).year
df['month'] = pd.DatetimeIndex(df["TX_DATE"]).month
df = df.drop("TX_DATE", axis=1)

### Making dummy variables

- gender
- abo{9}
- EXH_PERIT_ACCCESS{3}
- EXH_VASC_ACCESS{3}
- PREV_TX{2}
- PREV_KI_TX{2}
- MALIG_TRR{3}
- PERM_STATE_TRR{51}
- txkid{3}
- ABO_DON{9}
- DON_TY{3}
- GENDER_DON{2}
- HOME_STATE_DON{51}
- ABO_MAT{3}
- GRF_STAT_KI{2}
- DWFG_KI{2}
- TX_PROCEDUR_TY_KI{[did you get one or two? See data_sheet]}
- PRV_TX_ANY{2}
- PX_STAT{4}
- SHARE_TY{4}
- AGE_GROUP{2}
- malig{3} (previous malignancies)
- LT_ONE_WEEK_DON {2} (7 days old or less?)
- RECOV_OUT_US{2}

In [31]:
dummy_columns = ['gender', 'abo', 'EXH_PERIT_ACCESS', 'EXH_VASC_ACCESS', 'PREV_TX', 'PREV_KI_TX',
                 'MALIG_TRR', 'txkid', 'ABO_DON', 'DON_TY', 'GENDER_DON', 'DON_TY', 'GENDER_DON',
                 'HOME_STATE_DON', 'ABO_MAT', 'GRF_STAT_KI', 'DWFG_KI', 'PREV_TX_ANY', 'PX_STAT',
                 'SHARE_TY', 'AGE_GROUP', 'malig', 'LT_ONE_WEEK_DON', 'RECOV_OUT_US', 'year', 'month']
funky_columns = ['TX_PROCEDUR_TY_KI']

# Took out 'PERM_STATE'

In [32]:
df_non_dummy = df.drop(dummy_columns, axis=1)
df_non_dummy

,NUM_PREV_TX,A1,A2,B1,B2,DR1,DR2,REM_CD,END_STAT,INIT_AGE,...,AGE_DON,ETHCAT_DON,END_STAT_KI,age,GTIME_KI,GSTATUS_KI,DAYSWAIT_CHRON_KI,TX_PROCEDUR_TY_KI,pstatus,ptime
0,0,1,26,38,44,1,4,4,4010,38.0,...,35.0,1,4010.0,38,3678.0,1.0,34.0,102.0,0,4746.0
1,1,2,23,44,62,7,11,4,4010,43.0,...,17.0,1,4010.0,43,8634.0,0.0,97.0,102.0,0,8634.0
2,0,3,25,7,27,1,13,15,4010,42.0,...,44.0,1,4010.0,43,5219.0,1.0,247.0,101.0,0,5219.0
3,0,2,68,35,39,4,0,4,4010,9.0,...,16.0,1,4010.0,9,2098.0,1.0,35.0,102.0,0,2206.0
4,0,1,2,62,27,3,4,4,4010,25.0,...,16.0,1,4010.0,28,2248.0,1.0,1047.0,101.0,1,5141.0
5,0,2,11,35,62,1,2,4,4010,25.0,...,29.0,1,4010.0,26,5049.0,1.0,335.0,101.0,1,6459.0
6,0,3,19,17,70,11,7,4,4010,43.0,...,44.0,2,4010.0,45,237.0,1.0,575.0,101.0,1,1710.0
7,0,2,11,62,0,4,13,15,4010,45.0,...,66.0,1,4010.0,45,1459.0,0.0,37.0,101.0,1,5091.0
8,0,3,0,7,14,1,2,4,4010,75.0,...,64.0,1,4010.0,76,18.0,1.0,101.0,102.0,1,18.0
9,0,1,31,8,51,11,17,4,4010,65.0,...,27.0,1,4010.0,66,2635.0,1.0,321.0,101.0,1,2635.0


In [33]:
df_dummified = pd.get_dummies(df[dummy_columns])


In [34]:
df = df_non_dummy.merge(df_dummified, left_index=True, right_index=True)
df

,NUM_PREV_TX,A1,A2,B1,B2,DR1,DR2,REM_CD,END_STAT,INIT_AGE,...,AGE_GROUP_A,AGE_GROUP_P,malig_N,malig_U,malig_Y,LT_ONE_WEEK_DON_N,LT_ONE_WEEK_DON_Y,RECOV_OUT_US_,RECOV_OUT_US_N,RECOV_OUT_US_Y
0,0,1,26,38,44,1,4,4,4010,38.0,...,1,0,0,1,0,1,0,0,1,0
1,1,2,23,44,62,7,11,4,4010,43.0,...,1,0,0,1,0,1,0,0,1,0
2,0,3,25,7,27,1,13,15,4010,42.0,...,1,0,0,1,0,1,0,0,1,0
3,0,2,68,35,39,4,0,4,4010,9.0,...,0,1,0,1,0,1,0,0,1,0
4,0,1,2,62,27,3,4,4,4010,25.0,...,1,0,0,1,0,1,0,0,1,0
5,0,2,11,35,62,1,2,4,4010,25.0,...,1,0,0,1,0,1,0,0,1,0
6,0,3,19,17,70,11,7,4,4010,43.0,...,1,0,0,1,0,1,0,0,1,0
7,0,2,11,62,0,4,13,15,4010,45.0,...,1,0,0,1,0,1,0,0,1,0
8,0,3,0,7,14,1,2,4,4010,75.0,...,1,0,0,1,0,1,0,0,1,0
9,0,1,31,8,51,11,17,4,4010,65.0,...,1,0,0,1,0,1,0,0,1,0


In [15]:
df.columns

Index(['NUM_PREV_TX', 'A1', 'A2', 'B1', 'B2', 'DR1', 'DR2', 'PERM_STATE',
       'REM_CD', 'END_STAT',
       ...
       'AGE_GROUP_A', 'AGE_GROUP_P', 'malig_N', 'malig_U', 'malig_Y',
       'LT_ONE_WEEK_DON_N', 'LT_ONE_WEEK_DON_Y', 'RECOV_OUT_US_',
       'RECOV_OUT_US_N', 'RECOV_OUT_US_Y'],
      dtype='object', length=163)

In [35]:
y_variable = 'DAYSWAIT_CHRON_KI'
X_variables = list(df.columns[df.columns != y_variable])
X = df[X_variables].to_numpy()
y = df[y_variable].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y)

forest = RandomForestRegressor(n_estimators=5)
model = forest.fit(X_train, y_train)

In [36]:
model.score(X_test,y_test)

0.8235839009406067

In [38]:
num_21 =  X[21,:].copy().reshape(1, -1)
num_21.shape
print(model.predict(num_21))

[150.6]


In [40]:
rand_people = [68592, 360018, 300328, 173781, 311525, 336194, 298122, 343924, 22757, 18917]
results =[]
for i in rand_people:
    person = X[i,:].copy().reshape(1,-1)
    print(model.predict(person))

[748.]
[145.6]
[407.2]
[1070.4]
[684.2]
[1329.8]
[75.4]
[397.4]
[307.6]
[334.2]


In [54]:
rand_people = [68592, 360018, 300328, 173781, 311525, 336194, 298122, 343924, 22757, 18917]
times = [748.,145.6,407.2,1070.4,684.2,1329.8,75.4,397.4,307.6, 334.2]
times = np.array(times).reshape(1,-1)
rand_people_wl_time  = pd.DataFrame(wl_times, columns = rand_people)
rand_people_wl_time

In [55]:
times = np.array(times).reshape(1,-1)

In [56]:
rand_people_wl_time  = pd.DataFrame(wl_times, columns = rand_people)
rand_people_wl_time

,68592,360018,300328,173781,311525,336194,298122,343924,22757,18917
0,748.0,145.6,407.2,1070.4,684.2,1329.8,75.4,397.4,307.6,334.2
